In [1]:
import os
import re
from collections import defaultdict
import csv
from typing import Dict, Set, Tuple


def find_function_usage(directory: str) -> Tuple[Dict[str, Dict[str, int]], Set[str], Dict[str, str]]:
    """
    Find all defined functions in .py files within a directory and track their usage
    throughout the codebase.

    Returns:
        - A dictionary mapping function names to files and their usage count
        - A set of unused functions
        - A dictionary mapping function names to their source files
    """
    # Pattern to match Python function definitions
    function_pattern = re.compile(r'''
        ^\s*def\s+([a-zA-Z_]\w*)\s*\(  # Match 'def function_name('
        (?!.*(?:__init__|__call__|__[^_]+__))  # Exclude special methods
    ''', re.VERBOSE | re.MULTILINE)

    # File extensions to scan (only Python files)
    extensions = ('.py',)

    # To store all function names and their source files
    functions_map = {}  # name -> source file
    all_files = []

    # First pass: Collect all function definitions
    print("Scanning for function definitions...")
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(extensions):
                filepath = os.path.join(root, filename)
                relative_path = os.path.relpath(filepath, directory)
                all_files.append((relative_path, filepath))

                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()

                        for match in function_pattern.finditer(content):
                            func_name = match.group(1)
                            # Skip private functions (starting with _)
                            if not func_name.startswith('_'):
                                functions_map[func_name] = relative_path
                                print(f"Found function: {func_name} in {relative_path}")

                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

    print(f"Found {len(functions_map)} functions")

    # Second pass: Count occurrences of each function
    print("Scanning for usage...")
    usage_by_function = defaultdict(lambda: defaultdict(int))

    for relative_path, filepath in all_files:
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
                # Remove comments and strings to avoid false positives
                code_without_comments = re.sub(
                    r'#.*$|"""[\s\S]*?"""|\'\'\'[\s\S]*?\'\'\'|"(?:[^"\\]|\\.)*"|\'(?:[^\'\\]|\\.)*\'',
                    '',
                    content,
                    flags=re.MULTILINE
                )

                for func_name in functions_map.keys():
                    pattern = r'\b' + re.escape(func_name) + r'(?=\s*\()'  # Look for function calls
                    matches = re.findall(pattern, code_without_comments)
                    if matches:
                        usage_by_function[func_name][relative_path] = len(matches)

        except Exception as e:
            print(f"Error reading {filepath}: {e}")

    # Identify unused functions
    unused_functions = set()
    for func_name, usages in usage_by_function.items():
        source_file = functions_map.get(func_name)
        if not usages or (len(usages) == 1 and source_file in usages):
            unused_functions.add(func_name)

    return usage_by_function, unused_functions, functions_map


def write_results_to_csv(usage_data: Dict, unused_functions: Set, functions_map: Dict,
                         output_file: str = "function_usage_report.csv") -> None:
    """Write the results to a CSV file."""
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Function Name", "Source File", "Total Calls", "Unused?", "Found In Files"])

        sorted_functions = sorted(
            functions_map.keys(),
            key=lambda fn: sum(usage_data.get(fn, {}).values()),
            reverse=True
        )

        for func_name in sorted_functions:
            source_file = functions_map.get(func_name, "Unknown")
            usages = usage_data.get(func_name, {})
            total_uses = sum(usages.values())
            is_unused = "YES" if func_name in unused_functions else "NO"
            used_in = ", ".join(usages.keys()) if usages else "None"

            writer.writerow([func_name, source_file, total_uses, is_unused, used_in])

    print(f"Results written to {output_file}")


# Example usage
if __name__ == "__main__":
    directory = "."  # Current directory, change as needed
    usage_data, unused_funcs, funcs_map = find_function_usage(directory)
    write_results_to_csv(usage_data, unused_funcs, funcs_map)

Scanning for function definitions...
Found function: filter in webui_backend\config.py
Found function: run_migrations in webui_backend\config.py
Found function: load_json_config in webui_backend\config.py
Found function: save_to_db in webui_backend\config.py
Found function: reset_config in webui_backend\config.py
Found function: get_config in webui_backend\config.py
Found function: get_config_value in webui_backend\config.py
Found function: save_config in webui_backend\config.py
Found function: update in webui_backend\config.py
Found function: save in webui_backend\config.py
Found function: load_oauth_providers in webui_backend\config.py
Found function: google_oauth_register in webui_backend\config.py
Found function: microsoft_oauth_register in webui_backend\config.py
Found function: github_oauth_register in webui_backend\config.py
Found function: oidc_oauth_register in webui_backend\config.py
Found function: validate_cors_origins in webui_backend\config.py
Found function: validate_cor

In [ ]:
import re
import os


def remove_comments_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        code = file.read()

    # Remove block comments (/** ... */)
    code = re.sub(r'/\*\*.*?\*/', '', code, flags=re.DOTALL)

    # Remove single-line comments with space after // (// <space> ...)
    code = re.sub(r'//\s.*', '', code)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(code)


def remove_comments_from_repo(repo_path):
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.js', '.ts', '.java', '.cpp', '.c', '.cs', '.svelte')):  # Add more extensions if needed
                file_path = os.path.join(root, file)
                remove_comments_from_file(file_path)
                print(f"Processed: {file_path}")


# Example usage
repo_path = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"  # Change this to your repo path
remove_comments_from_repo(repo_path)

In [1]:
import re
import os


def convert_simple_block_comments_to_single_line(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        code = file.read()

    # Regular expression to match simple block comments (/** ... */) and capture content
    simple_block_comment_pattern = r'/\*\*\s*(\*?.*?)\s*\*/'

    # Function to replace simple block comments with // comments
    def replace_simple_block_comment(match):
        comment = match.group(1).strip()

        # Remove leading '*' characters and whitespace (if any)
        comment = re.sub(r'^\* ?', '', comment)  # Remove the '*' and optional space at the beginning of lines

        # If it's a one-line comment (without @tags), convert it to a single-line comment
        lines = comment.splitlines()
        if len(lines) == 1 and not lines[0].startswith('@'):
            return f"// {lines[0].strip()}"
        else:
            return match.group(0)  # Keep complex block comments unchanged

    code = re.sub(simple_block_comment_pattern, replace_simple_block_comment, code)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(code)


def convert_comments_in_repo(repo_path):
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(('.js', '.ts', '.java', '.cpp', '.c', '.cs', '.svelte')):  # Add more extensions if needed
                file_path = os.path.join(root, file)
                convert_simple_block_comments_to_single_line(file_path)
                print(f"Processed: {file_path}")


# Example usage
repo_path = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"  # Change this to your repo path
convert_comments_in_repo(repo_path)

Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\config\postcss.config.js
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\config\prepare-pyodide.js
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\config\svelte.config.js
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\config\tailwind.config.js
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\config\vite.config.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\app.d.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\constants.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\dayjs.js
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\index.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\apis\index.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\apis\audio\index.ts
Processed: C:\Users\harold.noble\Desktop\RIC\app\frontend\src\lib\apis\auths\index.ts
Processed: C:\Users\harold.noble\De

In [2]:
import os
import re
from typing import Set, List

# Configuration
BACKEND_DIR = r"C:\Users\harold.noble\Desktop\RIC\app\backend"  # Replace with your backend directory
FRONTEND_DIR = r"C:\Users\harold.noble\Desktop\RIC\app\frontend"  # Replace with your frontend directory
ENDPOINT_FILE_EXTENSIONS = (".py")  # Backend file extensions
FRONTEND_FILE_EXTENSIONS = (".ts", ".svelte")  # Frontend file extensions


def extract_endpoints_from_backend(directory: str) -> Set[str]:
    """
    Extract all API endpoints defined in the backend files.
    Supports Flask and FastAPI syntax (e.g., @app.route('/path') or @app.get('/path')).
    """
    endpoints = set()
    endpoint_patterns = [
        r"@app\.route\s*\(['\"](.+?)['\"]",  # Flask: @app.route("/path")
        r"@app\.(get|post|put|delete)\s*\(['\"](.+?)['\"]"  # FastAPI: @app.get("/path")
    ]

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(ENDPOINT_FILE_EXTENSIONS):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    for pattern in endpoint_patterns:
                        matches = re.findall(pattern, content)
                        for match in matches:
                            # Handle tuple from FastAPI (method, path) vs single match from Flask
                            endpoint = match[1] if isinstance(match, tuple) else match
                            # Normalize endpoint (remove trailing slashes, etc.)
                            endpoint = endpoint.strip("/").split("?")[0]
                            endpoints.add(endpoint)
    return endpoints


def find_endpoint_references_in_frontend(directory: str, endpoints: Set[str]) -> Set[str]:
    """
    Search frontend files for references to backend endpoints.
    Looks for URL strings that match the endpoints.
    """
    used_endpoints = set()

    # Create a regex pattern to match endpoint usage in fetch/axios calls
    endpoint_regex = r"(?:fetch|axios\.(?:get|post|put|delete))\s*\(['\"]/?(.+?)['\"]"

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(FRONTEND_FILE_EXTENSIONS):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    matches = re.findall(endpoint_regex, content)
                    for match in matches:
                        # Normalize the match to compare with backend endpoints
                        match = match.strip("/").split("?")[0]
                        if match in endpoints:
                            used_endpoints.add(match)

    return used_endpoints


def find_unused_endpoints(backend_dir: str, frontend_dir: str) -> List[str]:
    """
    Find API endpoints defined in the backend but not used in the frontend.
    """
    # Step 1: Get all defined endpoints from backend
    defined_endpoints = extract_endpoints_from_backend(backend_dir)
    print(f"Found {len(defined_endpoints)} defined endpoints: {defined_endpoints}")

    # Step 2: Find endpoints referenced in frontend
    used_endpoints = find_endpoint_references_in_frontend(frontend_dir, defined_endpoints)
    print(f"Found {len(used_endpoints)} used endpoints: {used_endpoints}")

    # Step 3: Compute unused endpoints
    unused_endpoints = defined_endpoints - used_endpoints
    return sorted(list(unused_endpoints))


def main():
    unused_endpoints = find_unused_endpoints(BACKEND_DIR, FRONTEND_DIR)
    if unused_endpoints:
        print("\nUnused API Endpoints:")
        for endpoint in unused_endpoints:
            print(f" - /{endpoint}")
    else:
        print("\nNo unused API endpoints found.")


if __name__ == "__main__":
    main()

Found 7 defined endpoints: {'api/chat/completed', 'api/config', 'api/models', 'api/changelog', 'api/tasks/stop/{task_id}', 'api/chat/actions/{action_id}', 'api/models/base'}
Found 0 used endpoints: set()

Unused API Endpoints:
 - /api/changelog
 - /api/chat/actions/{action_id}
 - /api/chat/completed
 - /api/config
 - /api/models
 - /api/models/base
 - /api/tasks/stop/{task_id}


In [3]:
import re
import os


def remove_comments_from_python_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        code = file.read()

    # Remove block comments (''' ... ''' or """ ... """)
    code = re.sub(r'(""".*?"""|\'\'\'.*?\'\'\')', '', code, flags=re.DOTALL)

    # Remove single-line comments (# ...)
    code = re.sub(r'#.*', '', code)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(code)


def remove_comments_from_python_repo(repo_path):
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                remove_comments_from_python_file(file_path)
                print(f"Processed: {file_path}")


# Example usage
repo_path = r"C:\Users\harold.noble\Desktop\RIC\app\backend"  # Change this to your repo path
remove_comments_from_python_repo(repo_path)

Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\config.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\constants.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\env.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\functions.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\main.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\tasks.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\__init__.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\internal\db.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\internal\wrappers.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\internal\migrations\001_initial_schema.py
Processed: C:\Users\harold.noble\Desktop\RIC\app\backend\webui_backend\internal\migrations\002_add_local_sharing.py
Processed: C:\Users\haro